In [5]:
import logging
logger = logging.getLogger(__name__)
logger.info("llm_handler logger")

from dotenv import load_dotenv
load_dotenv()
from config import CHROMA_DB_PATH, PROMPT_PATH

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from core.vector_utils import get_retriever, get_vectorestore, check_chroma_db_status

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic



In [3]:
with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()
print(f"✅ System prompt 로드 완료 (길이: {len(SYSTEM_PROMPT)} 문자)")

vectorestore = get_vectorestore()
print(f"✅ Chroma 벡터스토어 로드 완료 (경로: {CHROMA_DB_PATH})")

check_chroma_db_status(vectorestore)

retriever = get_retriever(vectorestore)
print("✅ Retriever 생성 완료")


✅ System prompt 로드 완료 (길이: 2576 문자)
✅ Chroma 벡터스토어 로드 완료 (경로: C:\workspace\Coding_Heroes_Server\data\chroma_db)
📊 Chroma DB 문서 개수: 4
✅ Retriever 생성 완료


In [4]:
# ChatPromptTemplate 정의
prompt_template = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    
    ("human", """
        스테이지 단계: {stage}
        
        참고 문서:
        {context}

        현재 사용자 질문: {user_question}
        
        위의 스테이지 정보와 참고 문서를 바탕으로 사용자의 질문에 대해 친절하고 도움이 되는 답변을 해주세요.
    """)
])


In [6]:
openai_llm = ChatOpenAI(model="gpt-4o", temperature=0)

anthropic_llm = ChatAnthropic(
    model="claude-sonnet-4-20250725",
    temperature=0,
)

In [10]:
llm = openai_llm

In [11]:


def format_docs(docs):
    context_str = "\n\n".join(doc.page_content for doc in docs)
    print("🔎 [RAG] context(문서 내용):\n", context_str)
    return context_str

def get_stage_query(input_dict):
    """스테이지 정보를 바탕으로 검색 쿼리 생성"""
    stage = input_dict["stage"]
    return f"스테이지 {stage}"


print("🔧 Chain 구성 시작...")
chain = (
    {
        "stage": RunnablePassthrough(),
        "user_question": RunnablePassthrough(),
        "context": RunnablePassthrough() | get_stage_query | retriever | format_docs,
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

🔧 Chain 구성 시작...


In [12]:
response = chain.invoke({
    "stage": "1",
    "user_question": "웜 어떻게 잡아"
})

🔎 [RAG] context(문서 내용):
 # 스테이지 목표
행동블록을 잘 조합합해서 **웜 한마리**를 퇴치하자!

- 난이도: 1/5
- 사용블록: 이동한다, 공격한다
- 유닛 종류: 백신 멍멍이

## 요구 조건

- 백신 멍멍이는 적을 만나면 **이동하고 공격해야** 해.
- 제어문, 조건, 반복 없이 **간단한 직선 흐름**을 사용해.
- 함수, 조건 블록은 사용하지 않는다.

## 자주 하는 실수

- 공격 블록만 넣고 이동하지 않음   
- 행동 순서가 반대 (공격 → 이동) 

## 정답 예시 (JSON)
```json
{
  "unit": "백신 멍멍이",
  "line": [
    {
      "block_id": 1,
      "type": "ACTION",
      "name": "이동한다",
      "next": 2
    },
    {
      "block_id": 2,
      "type": "ACTION",
      "name": "공격한다",
      "next": null
    }
  ],
  "custom_function": []
}

# 스테이지 목표
행동블록을 잘 조합합해서 **웜 한마리**를 퇴치하자!

- 난이도: 1/5
- 사용블록: 이동한다, 공격한다
- 유닛 종류: 백신 멍멍이

## 요구 조건

- 백신 멍멍이는 적을 만나면 **이동하고 공격해야** 해.
- 제어문, 조건, 반복 없이 **간단한 직선 흐름**을 사용해.
- 함수, 조건 블록은 사용하지 않는다.

## 자주 하는 실수

- 공격 블록만 넣고 이동하지 않음   
- 행동 순서가 반대 (공격 → 이동) 

## 정답 예시 (JSON)
```json
{
  "unit": "백신 멍멍이",
  "line": [
    {
      "block_id": 1,
      "type": "ACTION",
      "name": "이동한다",
      "next": 2
    },
    {
      "block_id": 2,
  

In [13]:
response

'스테이지 1에서는 백신 멍멍이를 사용해서 웜을 잡아야 해! 간단하게 "이동한다" 블록을 먼저 놓고, 그 다음에 "공격한다" 블록을 연결하면 돼. 순서가 중요하니까 꼭 이동 후 공격해야 해! 제어문이나 조건 없이 직선 흐름으로 만들어야 하니까, 블록 두 개만 사용하면 돼. 잘 해봐!'